#Paquetes necesarios

In [1]:
import cv2  
import math 
import numpy as np
import os
from ultralytics import YOLO
import easyocr



Desde cámara, detección con yolov8 y modelo nano

In [2]:
# Carga del modelo
model = YOLO('yolov8n.pt')

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Captura desde lawebcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Perform inference on an image
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confidence --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Class name -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()


0: 480x640 (no detections), 56.5ms
Speed: 2.5ms preprocess, 56.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

Confidence ---> 0.3
Class name --> bicycle
0: 480x640 1 bicycle, 46.5ms
Speed: 2.0ms preprocess, 46.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

Confidence ---> 0.26
Class name --> bicycle
0: 480x640 1 bicycle, 46.0ms
Speed: 1.0ms preprocess, 46.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 57.0ms
Speed: 1.5ms preprocess, 57.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

Confidence ---> 0.33
Class name --> clock
0: 480x640 1 clock, 42.0ms
Speed: 1.0ms preprocess, 42.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 44.0ms
Speed: 1.5ms preprocess, 44.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 43.0ms
Speed: 1.0ms preprocess, 43.0ms inference, 0.5ms postproc

Reconocimiento de caracteres tras instalar pytesseract y tesseract

In [8]:
# Carga del modelo
model = YOLO('yolov8n.pt')

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

folder_path = "./images/"

image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

reader = easyocr.Reader(['es'])

def encontrar_matricula(car_img):
    gray = cv2.cvtColor(car_img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 30, 150)

    contours, _ = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        rect = cv2.minAreaRect(cnt)  # Obtiene el rectángulo rotado más pequeño
        box = cv2.boxPoints(rect)  # Obtiene los cuatro puntos del rectángulo
        box = np.int0(box)  # Redondea los valores a enteros

        # Calcular el área del rectángulo para filtrar contornos pequeños
        area = cv2.contourArea(box)
        if 800 < area < 5000:
            # Extraer la porción de la imagen dentro del rectángulo
            mask = np.zeros_like(gray)
            cv2.drawContours(mask, [box], 0, 255, -1)
            masked = cv2.bitwise_and(car_img, car_img, mask=mask)
            x, y, w, h = cv2.boundingRect(cnt)
            crop = masked[y:y+h, x:x+w]

            # Convertir recorte a formato RGB para EasyOCR
            crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)

            # Aplicar OCR usando EasyOCR
            results = reader.readtext(crop_rgb)

            # Si se detecta texto, dibujar el rectángulo y agregar texto en la esquina inferior izquierda del rectángulo
            if results:
                # Agregar texto de matrícula en la esquina inferior izquierda del rectángulo
                matricula_text = results[0][-2]
                if len(matricula_text) >= 2:
                    cv2.drawContours(car_img, [box], 0, (0, 255, 0), 3)
                    return matricula_text

    return ""








                
"""
def encontrar_matricula(car_img):
    # Convertir a escala de grises
    gray = cv2.cvtColor(car_img, cv2.COLOR_BGR2GRAY)
    # Aplicar desenfoque para reducir el ruido
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    # Detector de bordes
    edged = cv2.Canny(blur, 30, 150)

    # Buscar contornos
    contours, _ = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    rectangulos = []

    for cnt in contours:
        # Aproximar el contorno a un polígono
        epsilon = 0.05 * cv2.arcLength(cnt, True)  # Aumenta el valor de epsilon para una forma más general
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        
        # Considerar contornos con un número de vértices cercano a 4 (por ejemplo, de 4 a 10)
        if 4 <= len(approx) <= 10:  
            area = cv2.contourArea(cnt)
            # Ajustar los umbrales de área para incluir un rango más amplio
            if 50 < area < 20000:  # Los umbrales dependen del tamaño esperado de los rectángulos
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = w / float(h)

                # Considerar un rango más amplio de proporciones de aspecto
                if 0.2 < aspect_ratio < 5:
                    rectangulos.append(cnt)
                    # Dibujar el contorno en la imagen recortada, no en la original
                    cv2.drawContours(car_img, [cnt], 0, (0, 255, 0), 3)
"""

for image_file in image_files:
    # Lee la imagen desde la carpeta
    img = cv2.imread(os.path.join(folder_path, image_file))
    target_size = (800, 600)
    img = cv2.resize(img, target_size)
    
    # Perform inference on an image
    results = model(img)

    # Para cada detección
    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Identificación de la matrícula para coches
            if classNames[int(box.cls[0])] == "car":
                # Dibuja un rectángulo negro alrededor del coche
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 2)
                # Calcular la mitad de la altura del recuadro
                mitad_altura = (y2 - y1) // 2

                # Recortar solo la mitad inferior del recuadro
                mitad_inferior_car_img = img[y1 + mitad_altura:y2, x1:x2]

                # Pasar esta mitad inferior a la función encontrar_matricula
                matricula_text= encontrar_matricula(mitad_inferior_car_img)

                cv2.putText(img, "car " + matricula_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                

        # Muestra la imagen con las detecciones
        cv2.imshow('Image', img)
        cv2.waitKey(0)

# Cierra la ventana al finalizar
cv2.destroyAllWindows()

"""
# Para cada detección
for r in results:
    boxes = r.boxes

    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
        # [ ... Tu código para dibujar el contenedor y clase ... ]

        # Identificación de la matrícula para coches
        if classNames[int(box.cls[0])] == "car":
            # Seleccionar la parte inferior del coche para buscar la matrícula
            car_img = img[y1:y2, x1:x2]

            # Aplica la función encontrar_matricula a la imagen del coche
            encontrar_matricula(car_img)

            # Dibujar el resultado de vuelta en la imagen original
            img[y1:y2, x1:x2] = car_img

# Muestra la imagen
cv2.imshow('Imagen Detectada', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""


0: 480x640 4 cars, 1 truck, 8.0ms
Speed: 3.0ms preprocess, 8.0ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)


C:\Users\ganma\AppData\Local\Temp\ipykernel_22312\1027782351.py:33: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)  # Redondea los valores a enteros



0: 480x640 1 person, 1 car, 8.5ms
Speed: 1.5ms preprocess, 8.5ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 1 truck, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640

'\n# Para cada detección\nfor r in results:\n    boxes = r.boxes\n\n    for box in boxes:\n        x1, y1, x2, y2 = box.xyxy[0]\n        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)\n        \n        # [ ... Tu código para dibujar el contenedor y clase ... ]\n\n        # Identificación de la matrícula para coches\n        if classNames[int(box.cls[0])] == "car":\n            # Seleccionar la parte inferior del coche para buscar la matrícula\n            car_img = img[y1:y2, x1:x2]\n\n            # Aplica la función encontrar_matricula a la imagen del coche\n            encontrar_matricula(car_img)\n\n            # Dibujar el resultado de vuelta en la imagen original\n            img[y1:y2, x1:x2] = car_img\n\n# Muestra la imagen\ncv2.imshow(\'Imagen Detectada\', img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [3]:
# Tesseract
import cv2
import pytesseract

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('toy.tif') 
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Aplica reconocedor a imagen cargada
print(pytesseract.image_to_string(img_rgb))

['eng', 'osd']
Hasta el infinito y mas alla



Reconocimiento decaracteres tras instalar easyocr

In [2]:


#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
result = reader.readtext('toy.tif')
print(result)

#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[49, 85], [617, 85], [617, 147], [49, 147]], 'Hasta el infinito y más allá', 0.6744627670805162)]


In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:

if __name__ == '__main__':
    # Initialize the model
    model = YOLO("yolov8n.yaml")

    # Train the model with adjusted settings
    results = model.train(data='C:/Users/ganma/Downloads/dataset/data.yaml', 
                          epochs=100, 
                          imgsz=800, 
                          plots=True, 
                          workers=0)  # Set workers to 0


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               


  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

train: Scanning C:\Users\ganma\Downloads\train\labels.cache... 1156 images, 64 backgrounds, 0 corrupt: 100%|██████████| 1156/1156 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 82, len(boxes) = 1772. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning C:\Users\ganma\Downloads\valid\labels.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]

...

    100/100      3.23G      1.131     0.7439      1.407          6        800: 100%|██████████| 73/73 [00:26<00:00,  2.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.60it/s]

                   all        118        204       0.89       0.73      0.795      0.414



100 epochs completed in 0.832 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 6.3MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics YOLOv8.0.215 🚀 Python-3.11.5 torch-2.1.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.51it/s]


                   all        118        204      0.885      0.716      0.797      0.422
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\train6


In [3]:
!yolo task=detect mode=val model="./runs/detect/train6/weights/best.pt" data="C:/Users/ganma/Downloads/dataset/data.yaml"

Ultralytics YOLOv8.0.215 🚀 Python-3.11.5 torch-2.1.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 11, len(boxes) = 204. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                   all        118        204      0.898      0.721      0.802      0.426
Speed: 1.9ms preprocess, 4.6ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs\detect\val
💡 Learn more at https://docs.ultralytics.com/modes/val



val: Scanning C:\Users\ganma\Downloads\valid\labels.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]
val: Scanning C:\Users\ganma\Downloads\valid\labels.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/8 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  12%|█▎        | 1/8 [00:04<00:33,  4.76s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 2/8 [00:05<00:13,  2.20s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  38%|███▊      | 3/8 [00:05<00:06,  1.36s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 4/8 [00:05<00:03,  1.08it/s]
                 Class     Images  Instances

In [5]:
!yolo task=detect mode=predict model="./runs/detect/train6/weights/best.pt" conf=0.2 source="./images/" save=True

Ultralytics YOLOv8.0.215 🚀 Python-3.11.5 torch-2.1.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\755934778_232384541_1706x960.jpg: 480x800 4 License-Plates, 74.5ms
image 2/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\Hyundai-Tucson-2021-web-1046x616.jpg: 480x800 1 License-Plate, 6.5ms
image 3/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\IMG_3864.jpg: 800x608 1 License-Plate, 77.5ms
image 4/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\IMG_3865.jpg: 800x608 2 License-Plates, 7.5ms
image 5/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\IMG_3866.jpg: 800x608 1 License-Plate, 7.0ms
image 6/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\c3.webp: 608x800 2 License-Plates, 91.0ms
image 7/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\c4.jpg: 544x800 (no detections), 72.5ms
image 8/12 c:\Users\ganma\Documents\VC\VC\VC\P5\images\coche1.jpg: 480x800 1 Li